# ETH-BTC Metrics

## Spec

## Load libraries

In [1]:
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
from pytz import timezone as timezone_conv
from datetime import timezone as timezone_info

import matplotlib.pyplot as plt
%matplotlib inline

## API + Config

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
from coinmetrics.api_client import CoinMetricsClient

client = CoinMetricsClient("https://community-api.coinmetrics.io/v4")

# or to use community API:
client = CoinMetricsClient()

In [4]:
assets = ['eth', 'btc']
asset_mapping = {i: assets[i] for i in range(len(assets))}
print(asset_mapping)

{0: 'eth', 1: 'btc'}


## Load assets 

In [5]:
asset_catalog = client.catalog_assets(assets=assets)
full_asset_catalog = client.catalog_full_assets(assets=assets)

In [6]:
print("*** catalog endpoint ***")
metrics = []  # Initialize an empty list to collect metric names
for asset_metadata in asset_catalog:
    asset_name = asset_metadata['asset']
    if asset_name.lower() == 'eth':  # Ensure case-insensitive comparison
        
        # Collect all available metric names for Ethereum
        for metric_info in asset_metadata['metrics']:
            metrics.append(metric_info['metric'])  # Add metric name to the list

        # Since we only want to process Ethereum, break after processing it
        break

*** catalog endpoint ***


In [7]:
metrics = [
    "AdrBalNtv0.01Cnt",
    "AdrBalNtv0.1Cnt",
    "AdrBalNtv1Cnt",
    "AdrBalNtv10Cnt",
    "BlkSizeMeanByte",
    "CapRealUSD",
    "FeeByteMeanNtv",
    "FlowInExNtv",
    "FlowOutExNtv",
    "FlowTfrFromExCnt",
    "NDF",
    "SplyAct1d",
    "SplyActPct1yr",
    "TxCnt",
    "VelCur1yr",
]


In [8]:
#Query API for prices, daily CM reference rates as dataframe
from datetime import datetime, timedelta
import logging
import pandas as pd
import pytz


# Set the query parameters
frequency = "1d"
start_time = "2019-01-01"
end_time = (datetime.now(pytz.timezone('US/Pacific')) + timedelta(days=1)).strftime('%Y-%m-%d')

# Log the operation
logging.info("Getting prices...")

# Fetch the data
df = client.get_asset_metrics(
    assets=['ETH', 'BTC'],
    metrics=metrics,
    frequency=frequency,
    start_time=start_time,
    end_time=end_time
).to_dataframe()

# Assign datatypes
df["time"] = pd.to_datetime(df["time"])
for metric in metrics:
    if df[metric].dtype == 'object':
        df[metric] = pd.to_numeric(df[metric], errors='coerce')

# Reshape dataset so assets are in columns, dates are the rows, and the values are metrics
df_pivot = df.pivot_table(
    index="time",
    columns="asset",
    values=metrics
)

# Adjust column names to include the asset prefix
df_pivot.columns = [f'{col[1]}_{col[0]}' for col in df_pivot.columns]

# Flatten the DataFrame for easier access if necessary
df_pivot.reset_index(inplace=True)

2024-03-01 08:45:00 INFO     Getting prices...
2024-03-01 08:45:07 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2024-03-01 08:45:19 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [9]:
df_pivot

,time,btc_AdrBalNtv0.01Cnt,eth_AdrBalNtv0.01Cnt,btc_AdrBalNtv0.1Cnt,eth_AdrBalNtv0.1Cnt,btc_AdrBalNtv10Cnt,eth_AdrBalNtv10Cnt,btc_AdrBalNtv1Cnt,eth_AdrBalNtv1Cnt,btc_BlkSizeMeanByte,...,btc_NDF,eth_NDF,btc_SplyAct1d,eth_SplyAct1d,btc_SplyActPct1yr,eth_SplyActPct1yr,btc_TxCnt,eth_TxCnt,btc_VelCur1yr,eth_VelCur1yr
0,2019-01-01 00:00:00+00:00,6457193.0,5725623.0,2539967.0,2553149.0,149476.0,240984.0,702986.0,948660.0,807949.261745,...,0.351609,0.635803,237928.302452,11221087.882036,47.132581,72.295246,235813.0,448168.0,15.225509,11.758691
1,2019-01-02 00:00:00+00:00,6458964.0,5729156.0,2541704.0,2554121.0,149284.0,241344.0,703693.0,948649.0,941637.291391,...,0.351823,0.636376,253147.284599,10343209.345505,47.049109,72.264375,271103.0,589959.0,15.171639,11.643999
2,2019-01-03 00:00:00+00:00,6459493.0,5733108.0,2542722.0,2555952.0,149740.0,240985.0,704873.0,948798.0,961764.425806,...,0.351586,0.636168,296672.502025,9498193.91421,46.936962,72.221087,287081.0,596620.0,15.131672,11.529535
3,2019-01-04 00:00:00+00:00,6463332.0,5732036.0,2543644.0,2553742.0,149680.0,240912.0,705051.0,948728.0,959605.516779,...,0.351335,0.636041,290548.531166,14212229.620577,46.868573,72.193628,283490.0,549398.0,15.087471,11.365511
4,2019-01-05 00:00:00+00:00,6474689.0,5730961.0,2545280.0,2549563.0,149760.0,240101.0,705233.0,944626.0,825104.368098,...,0.351538,0.637131,257436.803594,12886643.253894,46.718887,72.080216,269637.0,511189.0,15.021355,11.22179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,2024-02-25 00:00:00+00:00,12490326.0,24974699.0,4557653.0,5171159.0,154109.0,341634.0,1018266.0,1722695.0,1850330.256098,...,0.332416,0.681513,142335.133123,10462790.19109,31.13702,35.402746,421219.0,1102202.0,5.923488,6.025079
1882,2024-02-26 00:00:00+00:00,12500995.0,25005883.0,4562917.0,5171586.0,154120.0,341416.0,1017891.0,1722126.0,1721729.421053,...,0.332218,0.681429,252096.698348,12008606.912186,31.166858,35.529806,355313.0,1187460.0,5.926383,6.035505
1883,2024-02-27 00:00:00+00:00,12507564.0,25038901.0,4560700.0,5165150.0,154065.0,341194.0,1017845.0,1721422.0,1671326.682171,...,0.332562,0.682377,409249.657564,13464548.0962,31.22895,35.486583,388230.0,1219667.0,5.930415,6.042212
1884,2024-02-28 00:00:00+00:00,12513211.0,25066302.0,4558788.0,5160304.0,154089.0,340851.0,1017200.0,1719399.0,1798479.651007,...,0.332745,0.6825,625242.095592,13795537.630568,31.402772,35.379746,415406.0,1227000.0,5.940652,6.052825


In [10]:
# Convert 'time' from datetime to date
df_pivot['time'] = df_pivot['time'].dt.date

# Set 'time' as the index of the DataFrame
df_pivot.set_index('time', inplace=True)

# Now, df_pivot has 'time' as its index, and you can refer to it as df_final for clarity
df_final = df_pivot

# Display the first few rows of the final DataFrame
df_final.head()


,btc_AdrBalNtv0.01Cnt,eth_AdrBalNtv0.01Cnt,btc_AdrBalNtv0.1Cnt,eth_AdrBalNtv0.1Cnt,btc_AdrBalNtv10Cnt,eth_AdrBalNtv10Cnt,btc_AdrBalNtv1Cnt,eth_AdrBalNtv1Cnt,btc_BlkSizeMeanByte,eth_BlkSizeMeanByte,...,btc_NDF,eth_NDF,btc_SplyAct1d,eth_SplyAct1d,btc_SplyActPct1yr,eth_SplyActPct1yr,btc_TxCnt,eth_TxCnt,btc_VelCur1yr,eth_VelCur1yr
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01,6457193.0,5725623.0,2539967.0,2553149.0,149476.0,240984.0,702986.0,948660.0,807949.261745,13665.922841,...,0.351609,0.635803,237928.302452,11221087.882036,47.132581,72.295246,235813.0,448168.0,15.225509,11.758691
2019-01-02,6458964.0,5729156.0,2541704.0,2554121.0,149284.0,241344.0,703693.0,948649.0,941637.291391,18126.132355,...,0.351823,0.636376,253147.284599,10343209.345505,47.049109,72.264375,271103.0,589959.0,15.171639,11.643999
2019-01-03,6459493.0,5733108.0,2542722.0,2555952.0,149740.0,240985.0,704873.0,948798.0,961764.425806,19342.790323,...,0.351586,0.636168,296672.502025,9498193.91421,46.936962,72.221087,287081.0,596620.0,15.131672,11.529535
2019-01-04,6463332.0,5732036.0,2543644.0,2553742.0,149680.0,240912.0,705051.0,948728.0,959605.516779,18872.124595,...,0.351335,0.636041,290548.531166,14212229.620577,46.868573,72.193628,283490.0,549398.0,15.087471,11.365511
2019-01-05,6474689.0,5730961.0,2545280.0,2549563.0,149760.0,240101.0,705233.0,944626.0,825104.368098,17351.082028,...,0.351538,0.637131,257436.803594,12886643.253894,46.718887,72.080216,269637.0,511189.0,15.021355,11.22179


In [11]:
# Check if there are any NaN values in df_final
empty_values = df_final.isna().any()

# Display columns with empty values
print("Columns with empty values:")
print(empty_values[empty_values])


Columns with empty values:
Series([], dtype: bool)


In [12]:
print(len(df_final))
print(len(df_final.columns))


1886
30


In [13]:
import pandas as pd

# Assuming df_final is your DataFrame
# df_final = pd.read_csv('your_file.csv')  # or however you are getting your DataFrame

# Function to find and display duplicate column names
def show_duplicate_column_names(df):
    duplicate_columns = df.columns[df.columns.duplicated()]
    if len(duplicate_columns) > 0:
        print("Duplicate column names:")
        print(duplicate_columns)
    else:
        print("No duplicate column names.")

# Displaying the duplicate column names
show_duplicate_column_names(df_final)


No duplicate column names.


In [14]:
import pandas as pd

# Assuming df_final is your DataFrame
# df_final = pd.read_csv('your_file.csv')  # or however you are getting your DataFrame

# Function to check for duplicate column names in a DataFrame
def check_duplicate_column_names(df):
    if df_final.columns.duplicated().any():
        print("Duplicate column names found!")
        return True
    else:
        print("No duplicate column names.")
        return False

# Check for duplicate column names before saving to Parquet
#if not check_duplicate_column_names(df_final):
#    df_final.to_parquet('ETH-causal.parquet')


In [15]:
df_final.to_parquet('coin-metrics.parquet')